<a href="https://colab.research.google.com/github/summerolmstead/quantumworkshop/blob/main/Copy_of_BB84_Quantum_Key_Distribution_Protocol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BB84: The First Quantum Key Distribution Protocol
Learning Outcomes:

* Students will apply the principle of superposition and no-cloning theorem to construct a quantum key distribution protocol.
* Students will apply quantum operations and measurement principles to quantum key distribution.
* Students will understand the importance of using non-orthogonal states in QKD.
* Students will analyze the QKD protocol for detecting eavesdropping.

In [ ]:
%run pyfiles/bb84  #This runs the file that has the code
createTable()

Exception: File `'pyfiles/bb84.py'` not found.

# BB84 Simulator
The following cell provides widgets to set the values that control the main program.

* The first widget sets how many qubits are to be transmitted.
* The second widget controls whether Eve will be active or not.
* The third, fourth and fifth widgets set the bias of ✚ to ✖ for Alice, Bob and Eve.

These values are passed to the main program as parameters.

In [ ]:
# Code to generate GUI for entering program parameters
import ipywidgets as widgets
from IPython.display import display

# Configuration of the widget that controls the number of qubits
number_of_qubits = widgets.IntText(
                    value=100,
                    disabled=False
                    )
qubit_control = widgets.VBox([widgets.Label(value="Enter number of initial qubits"), number_of_qubits])

# Configuration of the widget that controls whether Eve is active or not
is_Eve_active =  widgets.ToggleButton(
                    value=False,
                    description='Click to activate',
                    disabled=False,
                    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
                    tooltip='Will Eve intercept the transmission',
                    icon=''
                    )
eve_message = widgets.Label(value="Eve is not active")
eve_control = widgets.VBox([eve_message, is_Eve_active])

# Configuration of an event handler for the eve control widget
def on_value_change(change):
    is_Eve_active.description = 'Click to deactivate' if change['new'] else 'Click to activate'
    eve_message.value = 'Eve is active' if change['new'] else 'Eve is not active'
is_Eve_active.observe(on_value_change, names='value')

# Configuration of the widget that controls Alice's bias towards the diagonal bases
abias = widgets.BoundedIntText(
                    value=50,
                    min=0,
                    max=100,
                    step=1,
                    disabled=False
                    )
abias_control = widgets.VBox([widgets.Label(value="Enter Alice's bias for diagnal basis over standard basis (0-100)"), abias])

# Configuration of the widget that controls Bob's bias towards the diagonal bases
bbias = widgets.BoundedIntText(
                    value=50,
                    min=0,
                    max=100,
                    step=1,
                    disabled=False
                    )
bbias_control = widgets.VBox([widgets.Label(value="Enter Bob's bias for diagnal basis over standard basis (0-100)"), bbias])

# Configuration of the widget that controls Eve's bias towards the diagonal bases
ebias = widgets.BoundedIntText(
                    value=50,
                    min=0,
                    max=100,
                    step=1,
                    disabled=False
                    )
ebias_control = widgets.VBox([widgets.Label(value="Enter Eve's bias for diagnal basis over standard basis (0-100)"), ebias])

# display the control widgets
display(qubit_control, eve_control, abias_control, bbias_control, ebias_control)

After setting the parameters you may either:

* Run each of the following cells one at a time in order or
* Select "Run All Below" in the "Cell" sub-menu from here

The following cells define the classes, function and main program. Output is provided after each cell.

In [ ]:
class Messenger:
    """Inherited class with methods common to both BB84 and B92 protocols"""
    from random import choice
    from random import choices

    # There are no class variables defined, only instance variables.
    # Therefore no data is shared between instances without going through the main program
    def __init__(self, number_of_qubits):
        """Initialize a new instance"""
        self.key = []
        self.bases = []
        self.valid_bit = []
        self.secret_bit = []
        for i in range(number_of_qubits):
            self.valid_bit.append(1)
            self.secret_bit.append(1)

    def generate_bits(self, number_of_bits):
        """Populate the key instance variable with a given number of random bits and return what was populated"""
        self.key = self.choices([0,1], k=number_of_bits)
        return self.key

    def generate_bases(self, number_of_qubits, biasToX):
        """Populate the bases instance variable with a given number of random bits and return what was populated"""
        self.bases = self.choices(['×','+'], cum_weights=[biasToX,100], k=number_of_qubits)
        return self.bases

    def scratch_bits(self, input):
        """Based on the markers given in the input parameter, clear the bits considered to be valid and secret"""
        markers = ['W', '?', '✓']
        for i, delete_indicator in enumerate(input):
            if delete_indicator in markers:
                self.valid_bit[i] = 0
                self.secret_bit[i] = 0
        return

    def choose_half(self, input):
        """Given which are valid choices in the input parameter, make a 50/50 choice which to mark from the valid choices"""
        output = []
        for i, bit in enumerate(input):
            if bit == 'W' or bit == '?':
                output.append('')
            else:
                if self.choice([0, 1]):
                    output.append('')
                else:
                    output.append('✓')
        return output

    def check_bits(self, input_key, bits_to_check, valid_bit):
        """From a given list of bits to check, see if the inputted bit equals the corresponding instance key bit"""
        output = []
        for i, bit in enumerate(bits_to_check):
            if bit == '✓':
                self.secret_bit[i] = 0
                if input_key[i] == self.key[i]:
                    output.append('✓')
                else:
                    output.append('')
            else:
                # If a bit is a valid bit but just not on the chosen list, reset it so it can be reused for multiple runs
                if valid_bit[i] != 'W' and valid_bit[i] != '?':
                    self.secret_bit[i] = 1
                output.append('')
        return output

    def show_valid_key(self):
        """Return the key for this instance based on the bits marked as valid"""
        output = []
        for key_bit, check_bit in zip(self.key, self.valid_bit):
            output.append(key_bit) if check_bit == 1 else output.append('')
        return output

    def show_secret_key(self):
        """Return the key for this instance based on the bits marked as secret"""
        output = []
        for key_bit, check_bit in zip(self.key, self.secret_bit):
            output.append(key_bit) if check_bit == 1 else output.append('')
        return output

class BB84Messenger(Messenger):
    """methods for BB84 messaging"""

    def encode_qubit(self, state, basis):
        """Return the qubit encoding based on the BB84 protocol"""
        vocabulary = [{'+':'→', '×':'↗'}, {'+':'↑', '×':'↖'}]
        return vocabulary[state][basis]

    def transmit_key(self):
        """Loop through all bits, returning the encoded BB84 qubits"""
        return ''.join([self.encode_qubit(key, basis) for key, basis in zip(self.key, self.bases)])

    def compare_bases(self, input):
        """Compare the inputted bases with the instance bases and return a 'C' if they match or 'W' if they do not"""
        output = []
        for my_basis, other_basis in zip(self.bases, input):
            output.append('C') if my_basis == other_basis else output.append('W')
        return output

    def receive_key(self, input):
        """Process an inputted string to extract the key bits and return what qubits were observed"""
        qubits = list(input)
        del self.key[:]
        observed = []
        for qubit, basis in zip(qubits, self.bases):
            if basis == '+' and qubit == '→':
                self.key.append(0)
                observed.append('→')
            elif basis == '×' and qubit == '↗':
                self.key.append(0)
                observed.append('↗')
            elif basis == '+' and qubit == '↑':
                self.key.append(1)
                observed.append('↑')
            elif basis == '×' and qubit == '↖':
                self.key.append(1)
                observed.append('↖')
            else:
                self.key.append(self.choice([0, 1]))
                observed.append(self.encode_qubit(self.key[len(self.key)-1], basis))
        return observed


def printTable(caption, rows):
    """Make a nice looking table like in the book"""
    from IPython.display import HTML, display
    display(HTML(
    '<table style="border-bottom: 4px double #333;border-top: 4px double #333;padding: 10px 0;">' +
    '<caption style="font-weight:900; color:black; border-top: 4px double #333;">' + caption + '</caption>' +
    '<tr>{}</tr></table>'.format('</tr><tr>'.join(
        '<td>' + row[0] + '</td><td>{}</td>'.format('</td><td>'.join(row[1])) for row in rows))))

# Main program

In [ ]:
# parameters passed into main program from GUI
n = number_of_qubits.value
eve_present = is_Eve_active.value
aliceXbias = abias.value
bobXbias = bbias.value
eveXbias = ebias.value

# instantiate a messenger class for each persona
alice84 = BB84Messenger(n)
bob84 = BB84Messenger(n)
eve84 = BB84Messenger(n)

# Let's see what we are working with
print("Program parameters have been entered into the program as follows:")
print("  * Alice will start with {} qubits to generate a key to send to Bob.".format(n))
print("  * Eve will {}intercept the transmission.".format('' if eve_present else 'not '))
print("  * Alice has a {}/{} chance of choosing diagonal basis over standard basis".format(aliceXbias, 100 - aliceXbias))
print("  * Bob has a {}/{} chance of choosing diagonal basis over standard basis".format(bobXbias, 100 - bobXbias))
print("  * Eve has a {}/{} chance of choosing diagonal basis over standard basis".format(eveXbias, 100 - eveXbias))

# Step 1 of the BB84 protocol - Alice sends a candidate key
* Alice flips a coin n times to determine which classical bits to send.
* She then flips the coin another n times to determine in which of the two bases to send those bits.
* She then sends the bits in their appropriate basis.

In [ ]:
# Alice randomly generates n bits.  These are private to Alice but pulled out here so they can be printed to show what was done
alice84bits = alice84.generate_bits(n)

# Alice randomly generates n bases biased towards X as defined by aliceXvalue choices out of 100
alice84bases = alice84.generate_bases(n, aliceXbias)

# Alice takes the bits she generated, encodes them into qubits using her generated bases, and puts them on the transmission line
transmission = alice84.transmit_key()

# Show all the work done by Alice in a nice convienient table
printTable(caption="Step 1: Alice sends " + str(n) + " random bits in random bases",
           rows = [["Bit&nbsp;number", map(str, range(1, n+1))],
                   ["Alice's&nbsp;random&nbsp;bits", map(str, alice84bits)],
                   ["Alice's&nbsp;random&nbsp;bases", alice84bases],
                   ["Alice&nbsp;sends", transmission]]
          )

# Step 2 of the BB84 protocol - Bob receives a candidate key
* Since Bob doesn't know the bases used by Alice, he flips a coin n times to determine the basis by which to measure.
* If Eve is present, then the transmission from Alice is intercepted then retransmitted to Bob
* Bob uses these random bases to measure the qubits sent from Alice

In [ ]:
# Bob randomly generates his n bases with a bias towards X defined by bobXbias
bob84bases = bob84.generate_bases(n, bobXbias)

# If eve is present, then the interception occurs
if eve_present:

    # Eve generates her bases with her given bias.
    eve84bases = eve84.generate_bases(n, eveXbias)

    # Eve intercepts the tranmission and reads it
    eve84received = eve84.receive_key(transmission)

    # She then replaces it with her own encoding from her set of randomly chosen bases
    transmission = eve84.transmit_key()

# Bob receives the transmission but it could have come from Alice or Eve
bob84received = bob84.receive_key(transmission)
if eve_present:
    # Show everything from Bob's point of view
    printTable(caption="Step 2: Bob receives " + str(n) + " random bits in random measurements",
               rows = [["Bit&nbsp;number", map(str, range(1, n+1))],
                       ["Eve's&nbsp;random&nbsp;bases", eve84bases],
                       ["Eve&nbsp;observes", eve84received],
                       ["Eve's&nbsp;bits", map(str, eve84.key)],
                       ["Bob's&nbsp;random&nbsp;bases", bob84bases],
                       ["Bob&nbsp;observes", bob84received],
                       ["Bob's&nbsp;bits", map(str, bob84.key)]])
else:
    # Show everything from Bob's point of view
    printTable(caption="Step 2: Bob receives " + str(n) + " random bits in random measurements",
               rows = [["Bit&nbsp;number", map(str, range(1, n+1))],
                       ["Bob's&nbsp;random&nbsp;bases", bob84bases],
                       ["Bob&nbsp;observes", bob84received],
                       ["Bob's&nbsp;bits", map(str, bob84.key)]])

## Step 3 of BB84 protocol - Alice and Bob evaluate the candidate key
* Bob tells Alice the bases that he chose
* Alice replies by telling Bob which basis is correct and which ones are wrong
* Alice and Bob scratch out the bits that had the wrong basis
* On average the length of the remaining subsequence should be about n/2 bits.

In [ ]:
# Bob sends his bases to Alice who does a comparison and returns which ones match correctly and which are wrong
bases_comparison = alice84.compare_bases(bob84bases)

# Alice drops the bits that do not match
alice84.scratch_bits(bases_comparison)

# So does Bob
bob84.scratch_bits(bases_comparison)

# Let's see how things are progressing
secret_key = bob84.show_valid_key()
if eve_present:
    printTable(caption="Step 3: Alice and Bob publicly compare bases used",
               rows = [["Bit&nbsp;number", map(str, range(1, n+1))],
                       ["Alice's&nbsp;random&nbsp;bases", alice84bases],
                       ["Eve's&nbsp;random&nbsp;bases", eve84bases],
                       ["Bob's&nbsp;random&nbsp;bases", bob84bases],
                       ["Which&nbsp;agree?", bases_comparison],
                       ["Shared&nbsp;secret&nbsp;bits", map(str, secret_key)]])
else:
    printTable(caption="Step 3: Alice and Bob publicly compare bases used",
               rows = [["Bit&nbsp;number", map(str, range(1, n+1))],
                       ["Alice's&nbsp;random&nbsp;bases", alice84bases],
                       ["Bob's&nbsp;random&nbsp;bases", bob84bases],
                       ["Which&nbsp;agree?", bases_comparison],
                       ["Shared&nbsp;secret&nbsp;bits", map(str, secret_key)]])

# and evaluate how we are doing so far
print('The expected number of matching bases is {} and the actual number is {}.'.format(int(n/2), bases_comparison.count('C')))

## Step 4 of the BB84 protocol - Alice and Bob check if there was an eavesdropper
* Bob randomly chooses half of the remaining n/2 bits and compares them with Alice.
* Assuming no channel errors, Alice's and Bob's comparison should match.
* If they do not match, then there was an eavesdropper and the whole set is discarded.

In [ ]:
alice_valid_key = alice84.show_valid_key()
bob_valid_key = bob84.show_valid_key()

# Bob chooses half of the bits from the correctly matched bases
bits_to_compare = bob84.choose_half(bases_comparison)

# Alice can check the chosen bits from her key against Bob's key
alice84.check_bits(bob_valid_key, bits_to_compare, bases_comparison)

# Bob can do the same against Alice's key and mark the bits that match
matched = bob84.check_bits(alice_valid_key, bits_to_compare, bases_comparison)

# Let's see how things turned out
shared_secret_key = bob84.show_secret_key()
printTable(caption="Step 4: Alice and Bob publicly compare half of the remaining bits",
           rows = [["Bit&nbsp;number", map(str, range(1, n+1))],
                   ["Bob's&nbsp;secret&nbsp;keys", map(str, bob_valid_key)],
                   ["Randomly&nbsp;chosen&nbsp;to&nbsp;compare", bits_to_compare],
                   ["Alice's&nbsp;secret&nbsp;keys", map(str, alice_valid_key)],
                   ["Which&nbsp;agree?", matched],
                   ["Unrevealed&nbsp;secret&nbsp;keys", map(str, shared_secret_key)]]
          )

# and evaluate our results
print('Alice and Bob matched {} out of {} bits.'.format(matched.count('✓'), bits_to_compare.count('✓')))

if matched.count('✓') == 0:
    print('There are no usable bits. You should choose different biases for your bases')
else:
    if matched.count('✓') == bits_to_compare.count('✓'):
        print('Everything seems OK, no eavesdropping detected')
        print('There are {} usable bits in the final key from {} shared bits and {} qubits initially transmitted'
              .format(len(list(filter(lambda x:x in {0,1}, shared_secret_key))), bases_comparison.count('C'), n))
        print('The key generation rate is {:.4f}'.format(len(list(filter(lambda x:x in {0,1}, shared_secret_key)))/n))
    else:
        print('Oh oh, there seems to be an eavesdropper')
        print('All bits must be discarded.  The key generation rate is 0.0000')